### Installs

In [1]:
!pip install pandas
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets
!pip install -q scann
!pip install ipywidgets

     |████████████████████████████████| 61kB 3.7MB/s 
     |████████████████████████████████| 394.7MB 45kB/s 
     |████████████████████████████████| 3.7MB 13.0MB/s 
     |████████████████████████████████| 11.1MB 1.4MB/s 


### Imports

In [5]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

### Data Set-up

In [19]:
# Ratings data.
ratings = tfds.load("movielens/100k-ratings", split="train")

In [20]:
for x in ratings.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}


In [21]:
dataframe = tfds.as_dataframe(ratings)
print(dataframe)

       bucketized_user_age     movie_genres  ... user_rating user_zip_code
0                     45.0              [7]  ...         4.0      b'53211'
1                     25.0          [4, 14]  ...         2.0      b'80525'
2                     18.0              [4]  ...         4.0      b'55439'
3                     50.0           [5, 7]  ...         4.0      b'06472'
4                     50.0         [10, 16]  ...         3.0      b'75094'
...                    ...              ...  ...         ...           ...
99995                 25.0       [0, 1, 15]  ...         4.0      b'80027'
99996                 35.0         [13, 16]  ...         4.0      b'60035'
99997                 18.0             [10]  ...         1.0      b'78264'
99998                 35.0  [0, 10, 15, 16]  ...         4.0      b'53210'
99999                 18.0              [4]  ...         2.0      b'95064'

[100000 rows x 12 columns]


In [24]:
# Pull out relevant columns and convert id columns to ints
relevant_cols = dataframe.loc[:, ['user_id', 'movie_id', 'user_rating', 'timestamp']]
relevant_cols['user_id'] = relevant_cols['user_id'].astype(int)
relevant_cols['movie_id'] = relevant_cols['movie_id'].astype(int)
print(relevant_cols)
print(relevant_cols.dtypes)

       user_id  movie_id  user_rating  timestamp
0          138       357          4.0  879024327
1           92       709          2.0  875654590
2          301       412          4.0  882075110
3           60        56          4.0  883326919
4          197       895          3.0  891409199
...        ...       ...          ...        ...
99995      774       228          4.0  888557237
99996      313       333          4.0  891012877
99997      262       567          1.0  879795430
99998      911       183          4.0  892839492
99999      276      1140          2.0  874791894

[100000 rows x 4 columns]
user_id          int64
movie_id         int64
user_rating    float64
timestamp        int64
dtype: object


In [25]:
relevant_cols['latest'] = relevant_cols.groupby(['user_id'])['timestamp'].rank(method='first', ascending=False)

train_ratings = relevant_cols[relevant_cols['latest'] != 1]
test_ratings = relevant_cols[relevant_cols['latest'] == 1]

train_ratings = train_ratings[['user_id', 'movie_id', 'user_rating']]
test_ratings = test_ratings[['user_id', 'movie_id', 'user_rating']]

print(train_ratings.shape)
print(test_ratings.shape)

(99057, 3)
(943, 3)


## Convert to Implicit Feedback Dataset

In [26]:
# Convert rating to 1 for everything to mark that the user has watched this item
train_ratings.loc[:, 'rating'] = 1
print(train_ratings)

       user_id  movie_id  user_rating  rating
0          138       357          4.0       1
1           92       709          2.0       1
2          301       412          4.0       1
3           60        56          4.0       1
4          197       895          3.0       1
...        ...       ...          ...     ...
99995      774       228          4.0       1
99996      313       333          4.0       1
99997      262       567          1.0       1
99998      911       183          4.0       1
99999      276      1140          2.0       1

[99057 rows x 4 columns]


In [27]:
# Add negative samples
all_movies = relevant_cols['movie_id'].unique()

users, items, labels = [], [], []
user_item_set = set(zip(train_ratings['user_id'], train_ratings['user_id']))
num_negatives = 4

for (u, i) in user_item_set:
  users.append(u)
  items.append(i)
  labels.append(1)
  for _ in range(num_negatives):
    negative_item = np.random.choice(all_movies)
    while (u, negative_item) in user_item_set:
      negative_item = np.random.choice(all_movies)
    users.append(u)
    items.append(negative_item)
    labels.append(0)

## Create Dataset

In [28]:
train_ds = tf.data.Dataset.from_tensor_slices((users, items, labels))

In [29]:
train_ds = train_ds.shuffle(1000)
for elem in train_ds.take(10):
  print(elem)

(<tf.Tensor: shape=(), dtype=int32, numpy=857>, <tf.Tensor: shape=(), dtype=int32, numpy=857>, <tf.Tensor: shape=(), dtype=int32, numpy=1>)
(<tf.Tensor: shape=(), dtype=int32, numpy=291>, <tf.Tensor: shape=(), dtype=int32, numpy=670>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)
(<tf.Tensor: shape=(), dtype=int32, numpy=277>, <tf.Tensor: shape=(), dtype=int32, numpy=514>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)
(<tf.Tensor: shape=(), dtype=int32, numpy=399>, <tf.Tensor: shape=(), dtype=int32, numpy=274>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)
(<tf.Tensor: shape=(), dtype=int32, numpy=554>, <tf.Tensor: shape=(), dtype=int32, numpy=1384>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)
(<tf.Tensor: shape=(), dtype=int32, numpy=441>, <tf.Tensor: shape=(), dtype=int32, numpy=1391>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)
(<tf.Tensor: shape=(), dtype=int32, numpy=168>, <tf.Tensor: shape=(), dtype=int32, numpy=1154>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)
(<tf.Tensor: shap